In [4]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import seaborn as sns
from ExplainableMaximumLikelihoodCalculator import ExplainableMaximumLikelihoodCalculator
import pysam
from Bio import SeqIO, Seq, SeqRecord, pairwise2
from Bio.pairwise2 import format_alignment
from LoadDataUtils import getListOfReadsFromBamFile, getListOfReadsFromFastaFile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math
import shap
from scipy.stats import chisquare
from scipy.special import rel_entr
fileNameSapiens =  "data/simulatedData/human_chinese_AF346973_500samples.fas" #File that contains 500 reads from homo sapiens
fileNameNeanderthals =  "data/simulatedData/Neanderthal_Goyet_KX198085_500samples.fas" #File that contains 500 reads from neanderthal
fileNameDenisovans =  "data/simulatedData/denisova_kx663333_500samples.fas" #File that contains 500 reads from denisovan
neanderthals_500_generated = getListOfReadsFromFastaFile(fileNameNeanderthals)
sapiens_500_generated = getListOfReadsFromFastaFile(fileNameSapiens)
denisovan_500_samples = getListOfReadsFromFastaFile(fileNameDenisovans)
path_to_frequencies_table = "data/substitution_matrix.tsv"
import itertools
 
def findsubsets(s, n):
    subsets = list(itertools.combinations(s, n))
    return [list(a) for a in subsets]

In [5]:
sapiens_reference_file_names = [
                    "data/reference_files/human_AF346981_French.fa",
                     "data/reference_files/human_AY195760_Korea.fa",
                      "data/reference_files/human_AY882416_Ethiopia.fa",
                      "data/reference_files/human_AY963586_Italian.fa",
                      "data/reference_files/human_AY195781_Caucasian.fa",
                      "data/reference_files/human_AY195757_Iraqi-Israeli.fa",
                      "data/reference_files/human_AY195749_NativeAmerican.fa"]
neanderthals_reference_file_names = [
                            "data/reference_files/neanderthal_mezmaiskaya1_FM865411.fa",
                           "data/reference_files/Neanderthal_Altai_KC879692.fa",
                           "data/reference_files/Neanderthal_Denisova11_full_mtDNA_KU131206.fa",
                           "data/reference_files/Neanderthal_Spy_94a_MG025538.fa",
                            "data/reference_files/Neanderthal_Vindija33.16_AM948965.fa",
                            "data/reference_files/Neanderthal_Vindija33.19_KJ533545.fa"]
denisovan_reference_file_names = [  
                        "data/reference_files/Denisova_MT576653.1.fa",
                        "data/reference_files/Denisova_MT576652.1.fa",
                        "data/reference_files/Denisova_4_FR695060.fa",
                        "data/reference_files/Denisova_8_KT780370.fa",
                        "data/reference_files/Denisova_manual_phalanx_NC_013993.fa",
                        "data/reference_files/Denisova_MT576651.1.fa"]

In [6]:
def generate_sample(sap, nean, den):
    list_before =  list(np.random.choice(sapiens_500_generated, sap)) + list(np.random.choice(neanderthals_500_generated, nean)) + list(np.random.choice(denisovan_500_samples, den))
    l = []
    for i in list_before:
        l.append(str(i))
    return l

In [62]:
def getEstimatorOnsample(size_of_sample):
    number_of_sap = random.randint(0,size_of_sample)
    number_of_nean = ((size_of_sample-number_of_sap)*2)//3
    number_of_denisovan = size_of_sample - number_of_sap - number_of_nean
    print(number_of_sap, number_of_nean, number_of_denisovan)
    sample = generate_sample(number_of_sap, number_of_nean, number_of_denisovan)
    maximum_likelihood_calculator_on_sample = ExplainableMaximumLikelihoodCalculator(sample,
                                                ref_neanderthal_file_names=neanderthals_reference_file_names,
                                                ref_sapien_file_names=sapiens_reference_file_names,
                                                ref_denisovan_file_names=denisovan_reference_file_names,
                                                path_to_substitution_matrix=path_to_frequencies_table,
                                                number_of_jobs=-1)
    return maximum_likelihood_calculator_on_sample

In [63]:
def getStability(ml_estimator, number_of_exps):
    estimation = ml_estimator.estimate_species_proportions_gradient_descent(3)
    diffs = []
    for i in range(number_of_exps):
        val = ml_estimator.estimate_species_proportions_gradient_descent(3).values
        diff = np.abs((estimation.values - val)).sum()
        diffs.append(diff)
    return diffs
            
    
    

In [65]:
stabilities

[0.020000000000000025,
 0.020000000000000254,
 1.1657341758564144e-15,
 1.4432899320127035e-15,
 1.4432899320127035e-15,
 1.3600232051658168e-15,
 0.02000000000000003,
 1.3426759704060487e-15,
 0.019999999999999754,
 1.609823385706477e-15,
 1.609823385706477e-15,
 1.3322676295501878e-15,
 1.124100812432971e-15,
 0.019999999999999976,
 1.4432899320127035e-15,
 1.27675647831893e-15,
 1.2212453270876722e-15,
 1.1102230246251565e-15,
 9.43689570931383e-16,
 0.0,
 9.43689570931383e-16,
 1.2490009027033011e-15,
 1.1102230246251565e-15,
 1.3877787807814457e-15,
 1.5543122344752192e-15,
 0.02000000000000003,
 1.3600232051658168e-15,
 1.0824674490095276e-15,
 0.019999999999999754,
 0.019999999999999366]

In [89]:
sorted(stabilities)

[0.0,
 9.43689570931383e-16,
 9.43689570931383e-16,
 1.0824674490095276e-15,
 1.1102230246251565e-15,
 1.1102230246251565e-15,
 1.124100812432971e-15,
 1.1657341758564144e-15,
 1.2212453270876722e-15,
 1.2490009027033011e-15,
 1.27675647831893e-15,
 1.3322676295501878e-15,
 1.3426759704060487e-15,
 1.3600232051658168e-15,
 1.3600232051658168e-15,
 1.3877787807814457e-15,
 1.4432899320127035e-15,
 1.4432899320127035e-15,
 1.4432899320127035e-15,
 1.5543122344752192e-15,
 1.609823385706477e-15,
 1.609823385706477e-15,
 0.019999999999999366,
 0.019999999999999754,
 0.019999999999999754,
 0.019999999999999976,
 0.020000000000000025,
 0.02000000000000003,
 0.02000000000000003,
 0.020000000000000254]

In [86]:
10**-15

1e-15

In [64]:
number_of_experiments = 30
sample_size = 30
stabilities = []
for i in range(number_of_experiments):
    print(i)
    estimator = getEstimatorOnsample(sample_size)
    list_of_diffs = getStability(estimator, 100)
    maximal_diff = np.max(list_of_diffs)
    stabilities.append(maximal_diff)
    print(maximal_diff)

0
0 20 10
Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  30
start working on read number 0
0.020000000000000025
1
0 20 10
Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  30
start working on read number 0
0.020000000000000254
2
2 18 10
Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  30
start working on read number 0
1.1657341758564144e-15
3
15 10 5
Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  30
start working on read number 0
1.4432899320127035e-15
4
16 9 5
Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  30
start working on read number 0
1.4432899320127035e-15
5
17 8 5
Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  30
start working on